## STEPS TO BE FOLLOWED

# 
1.IMPORTING NECCESARY LIBRARIES

2.CREATING S3 BUCKET

3.MAAPING Train and Test data in s3

4.Mapping the path of the modules in s3


In [4]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input,Session

In [61]:
bucket_name='bankapplif'
my_region=boto3.session.Session().region_name

In [62]:
print(my_region)

ap-south-1


In [63]:
##creating bucket we also manually do that

In [64]:
s3=boto3.resource('s3')
try:
    if  my_region == 'ap-south-1':
        s3.create_bucket(Bucket=bucket_name)
    print('created bucket')
except Exception as e:
    print('s3 error:',e)

s3 error: An error occurred (IllegalLocationConstraintException) when calling the CreateBucket operation: The unspecified location constraint is incompatible for the region specific endpoint this request was sent to.


In [65]:
### setting an output pathwhere the trained model would be saved

In [66]:
prefix='xg-boost-as-an-builtin-algo'
op_path='s3://{}//{}/ouput'.format(bucket_name,prefix)

In [67]:
print(op_path)

s3://bankapplif//xg-boost-as-an-builtin-algo/ouput


### downloading the dataset and storing in s3\


In [68]:
import pandas as pd
import urllib 

In [69]:
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

Success: downloaded bank_clean.csv.


In [70]:
try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


In [71]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [76]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], 
          axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [84]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], 
          axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

### building and training the algorithm

In [89]:
container=get_image_uri(boto3.Session().region_name,'xgboost',repo_version='1')
hyperparameters={'max_depth':'5',
                       'eta':'0.2',
                       'gamma':'4',
                       'min_child_weight':'6',
                       'subsample':'0.8',
                       'silent':'0',
                       'objective':'binary:logistic',
                       'num_round':'100'}

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [99]:
estimator = sagemaker.estimator.Estimator(image_uri=container,
                                            hyperparameters=  hyperparameters
                                          ,role=sagemaker.get_execution_role()
                                           , instance_count=1
                                           , instance_type='ml.m4.xlarge',
                                          train_volume_size=5
                                           ,output_path=op_path
                                           ,train_use_spot_instances=True
                                          ,train_max_run=300
                                         ,train_max_wait=600)

train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [100]:
estimator.fit({'train':s3_input_train,'validation':s3_input_test})

2022-07-11 12:40:17 Starting - Starting the training job...
2022-07-11 12:40:41 Starting - Preparing the instances for trainingProfilerReport-1657543217: InProgress
.........
2022-07-11 12:42:01 Downloading - Downloading input data...
2022-07-11 12:42:41 Training - Downloading the training image......
2022-07-11 12:43:42 Uploading - Uploading generated training modelArguments: train
[2022-07-11:12:43:34:INFO] Running standalone xgboost training.
[2022-07-11:12:43:34:INFO] File size need to be processed in the node: 2.9mb. Available memory size in the node: 8451.18mb
[2022-07-11:12:43:34:INFO] Determined delimiter of CSV input is ','
[12:43:34] S3DistributionType set as FullyReplicated
[12:43:34] 12357x59 matrix with 729063 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-07-11:12:43:34:INFO] Determined delimiter of CSV input is ','
[12:43:34] S3DistributionType set as FullyReplicated
[12:43:34] 12357x59 matrix with 729063 entries loaded from /opt

### deploying machine learning model

In [101]:
predictor=estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

------!

### prediction of the data

In [102]:
from sagemaker.serializers import CSVSerializer

In [105]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
predictor.serializer = CSVSerializer() # set the serializer type
predictions = predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [108]:
predictions_array

array([0.08000426, 0.03153157, 0.08322722, ..., 0.06332904, 0.04867871,
       0.04440598])

In [110]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 90.4%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10796)    27% (140)
Purchase        9% (1049)     73% (372) 



### deleting the endpoints

In [111]:
predictor.delete_endpoint(delete_endpoint_config=True)

In [112]:
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'F7707GYWVTVTQZBS',
   'HostId': 'Ar7GCWCpdfwylq4MlOQ0QfLmPyoW0B5bXfI1+O4sHR04gsIeFiDI2dCXgJSjAtSSS3MxNDHxdc0=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'Ar7GCWCpdfwylq4MlOQ0QfLmPyoW0B5bXfI1+O4sHR04gsIeFiDI2dCXgJSjAtSSS3MxNDHxdc0=',
    'x-amz-request-id': 'F7707GYWVTVTQZBS',
    'date': 'Mon, 11 Jul 2022 13:03:21 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': '/xg-boost-as-an-builtin-algo/ouput/xgboost-2022-07-11-12-32-07-905/profiler-output/system/incremental/2022071112/1657542900.algo-1.json'},
   {'Key': 'xg-boost-as-an-builtin-algo/test/test.csv'},
   {'Key': 'xg-boost-as-an-builtin-algo/train/train.csv'},
   {'Key': '/xg-boost-as-an-builtin-algo/ouput/xgboost-2022-07-11-12-40-17-155/profiler-output/system/training_job_end.ts'},
   {'Key': '/xg-boost-as-an-builtin-algo/ouput/xgboost-2022-07-11-1

### finish man....